In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os

# Download the dataset
path = kagglehub.dataset_download("salvatoresaia/ev-charging-stations-us")

print("Path to dataset files:", path)

files = os.listdir(path)
print("\nAvailable files:")
for file in files:
    print(f"  - {file}")

Path to dataset files: /Users/anirudhannabathula/.cache/kagglehub/datasets/salvatoresaia/ev-charging-stations-us/versions/2

Available files:
  - EV_Charging_Stations_Feb82024.xlsx
  - EV_Charging_Stations_Jan312023.xlsx


In [2]:
# Load the February 2024 dataset (most recent)
ev_charging_station_feb = os.path.join(path, 'EV_Charging_Stations_Feb82024.xlsx')

# Read the Excel file
df_feb = pd.read_excel(ev_charging_station_feb)

print("February 2024 EV Charging Stations Dataset Loaded Successfully!")
print(f"\nDataset shape: {df_feb.shape}")
print(f"Columns: {list(df_feb.columns)}")
print("\nFirst 5 records:")
print(df_feb.head())
print("\nData types:")
print(df_feb.dtypes)


February 2024 EV Charging Stations Dataset Loaded Successfully!

Dataset shape: (65134, 13)
Columns: ['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Network', 'EV Connector Types', 'Access Code', 'Access Detail Code', 'Facility Type']

First 5 records:
                        Station Name      Street Address         City State  \
0           LADWP - Truesdale Center  11797 Truesdale St   Sun Valley    CA   
1      Los Angeles Convention Center  1201 S Figueroa St  Los Angeles    CA   
2      LADWP - John Ferraro Building       111 N Hope St  Los Angeles    CA   
3         LADWP - Haynes Power Plant       6801 E 2nd St   Long Beach    CA   
4  LADWP - Harbor Generating Station    161 N Island Ave   Wilmington    CA   

     ZIP  EV Level1 EVSE Num  EV Level2 EVSE Num  EV DC Fast Count  \
0  91352                 NaN                57.0               2.0   
1  90015                 NaN                 7.0     

In [3]:
# Check for missing values in February dataset
print("Missing values per column:")
print(df_feb.isnull().sum())
print(f"\nTotal missing values: {df_feb.isnull().sum().sum()}")
print(f"\nPercentage of missing values per column:")
print((df_feb.isnull().sum() / len(df_feb) * 100).round(2))

Missing values per column:
Station Name              3
Street Address           35
City                      5
State                    14
ZIP                       1
EV Level1 EVSE Num    64467
EV Level2 EVSE Num     8457
EV DC Fast Count      55770
EV Network                2
EV Connector Types       36
Access Code               0
Access Detail Code    60210
Facility Type         47537
dtype: int64

Total missing values: 236537

Percentage of missing values per column:
Station Name           0.00
Street Address         0.05
City                   0.01
State                  0.02
ZIP                    0.00
EV Level1 EVSE Num    98.98
EV Level2 EVSE Num    12.98
EV DC Fast Count      85.62
EV Network             0.00
EV Connector Types     0.06
Access Code            0.00
Access Detail Code    92.44
Facility Type         72.98
dtype: float64


In [4]:
# Data Cleaning: Fill NaN values with 0 for charger counts (February dataset)
# NaN in charger columns means that charger type is not available at that station

print("Cleaning February 2024 charger count columns...")
print("\nBefore cleaning:")
print(f"EV Level1 EVSE Num - NaN count: {df_feb['EV Level1 EVSE Num'].isnull().sum()}")
print(f"EV Level2 EVSE Num - NaN count: {df_feb['EV Level2 EVSE Num'].isnull().sum()}")
print(f"EV DC Fast Count - NaN count: {df_feb['EV DC Fast Count'].isnull().sum()}")

# Fill NaN with 0 for charger counts (NaN means no chargers of that type)
df_feb['EV Level1 EVSE Num'] = df_feb['EV Level1 EVSE Num'].fillna(0)
df_feb['EV Level2 EVSE Num'] = df_feb['EV Level2 EVSE Num'].fillna(0)
df_feb['EV DC Fast Count'] = df_feb['EV DC Fast Count'].fillna(0)

print("\nAfter cleaning:")
print(f"EV Level1 EVSE Num - NaN count: {df_feb['EV Level1 EVSE Num'].isnull().sum()}")
print(f"EV Level2 EVSE Num - NaN count: {df_feb['EV Level2 EVSE Num'].isnull().sum()}")
print(f"EV DC Fast Count - NaN count: {df_feb['EV DC Fast Count'].isnull().sum()}")

# Calculate total chargers per station
df_feb['Total_Chargers'] = df_feb['EV Level1 EVSE Num'] + df_feb['EV Level2 EVSE Num'] + df_feb['EV DC Fast Count']

print(f"\nTotal chargers column created!")
print(f"Stations with at least one charger: {(df_feb['Total_Chargers'] > 0).sum()}")

Cleaning February 2024 charger count columns...

Before cleaning:
EV Level1 EVSE Num - NaN count: 64467
EV Level2 EVSE Num - NaN count: 8457
EV DC Fast Count - NaN count: 55770

After cleaning:
EV Level1 EVSE Num - NaN count: 0
EV Level2 EVSE Num - NaN count: 0
EV DC Fast Count - NaN count: 0

Total chargers column created!
Stations with at least one charger: 65128


In [5]:
# Analyze charger type distribution (February dataset)
print("Charger Type Distribution:")
print(f"\nStations with Level 1 chargers: {(df_feb['EV Level1 EVSE Num'] > 0).sum()}")
print(f"Stations with Level 2 chargers: {(df_feb['EV Level2 EVSE Num'] > 0).sum()}")
print(f"Stations with DC Fast chargers: {(df_feb['EV DC Fast Count'] > 0).sum()}")

print("\n\nCharger Statistics:")
print(df_feb[['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'Total_Chargers']].describe())

# Create charger type categories
df_feb['Has_Level1'] = df_feb['EV Level1 EVSE Num'] > 0
df_feb['Has_Level2'] = df_feb['EV Level2 EVSE Num'] > 0
df_feb['Has_DC_Fast'] = df_feb['EV DC Fast Count'] > 0

print("\n\nStations by charger availability:")
print(f"Level 1 only: {((df_feb['Has_Level1']) & (~df_feb['Has_Level2']) & (~df_feb['Has_DC_Fast'])).sum()}")
print(f"Level 2 only: {((~df_feb['Has_Level1']) & (df_feb['Has_Level2']) & (~df_feb['Has_DC_Fast'])).sum()}")
print(f"DC Fast only: {((~df_feb['Has_Level1']) & (~df_feb['Has_Level2']) & (df_feb['Has_DC_Fast'])).sum()}")
print(f"Multiple types: {((df_feb['Has_Level1'].astype(int) + df_feb['Has_Level2'].astype(int) + df_feb['Has_DC_Fast'].astype(int)) > 1).sum()}")

Charger Type Distribution:

Stations with Level 1 chargers: 667
Stations with Level 2 chargers: 56677
Stations with DC Fast chargers: 9364


Charger Statistics:
       EV Level1 EVSE Num  EV Level2 EVSE Num  EV DC Fast Count  \
count        65134.000000        65134.000000      65134.000000   
mean             0.044846            2.106120          0.604677   
std              0.968942            3.160297          2.419990   
min              0.000000            0.000000          0.000000   
25%              0.000000            1.000000          0.000000   
50%              0.000000            2.000000          0.000000   
75%              0.000000            2.000000          0.000000   
max             90.000000          338.000000         84.000000   

       Total_Chargers  
count    65134.000000  
mean         2.755642  
std          3.831305  
min          0.000000  
25%          2.000000  
50%          2.000000  
75%          2.000000  
max        350.000000  


Stations by charg

In [6]:
# Analyze network and access patterns (February dataset)
print("Network Distribution:")
print(df_feb['EV Network'].value_counts().head(10))

print("\n\nAccess Type Distribution:")
print(df_feb['Access Code'].value_counts())

print("\n\nFacility Type Distribution:")
print(df_feb['Facility Type'].value_counts().head(15))

print("\n\nState Distribution (Top 10):")
print(df_feb['State'].value_counts().head(10))

Network Distribution:
EV Network
ChargePoint Network    33918
Non-Networked           9180
Blink Network           5528
Tesla Destination       4227
Tesla                   2157
Volta                   1455
EV Connect              1301
Electrify America        940
eVgo Network             937
AMPUP                    855
Name: count, dtype: int64


Access Type Distribution:
Access Code
public     61308
private     3826
Name: count, dtype: int64


Facility Type Distribution:
Facility Type
HOTEL              3009
CAR_DEALER         2714
PARKING_LOT         950
OFFICE_BLDG         949
PUBLIC              887
FED_GOV             856
MUNI_GOV            705
SHOPPING_CENTER     634
COLLEGE_CAMPUS      523
PARKING_GARAGE      496
UTILITY             492
RESTAURANT          421
PAY_GARAGE          406
GAS_STATION         381
INN                 365
Name: count, dtype: int64


State Distribution (Top 10):
State
CA    16455
NY     3975
FL     3433
TX     3190
MA     3048
WA     2246
CO     2205


In [7]:
# Filter for public stations only (most relevant for users) - February dataset
df_feb_public = df_feb[df_feb['Access Code'] == 'public'].copy()

print(f"Total stations: {len(df_feb)}")
print(f"Public stations: {len(df_feb_public)} ({len(df_feb_public)/len(df_feb)*100:.1f}%)")
print(f"Private stations: {len(df_feb) - len(df_feb_public)} ({(len(df_feb) - len(df_feb_public))/len(df_feb)*100:.1f}%)")

print("\n\nPublic Stations - Charger Type Distribution:")
print(f"Stations with Level 1: {(df_feb_public['EV Level1 EVSE Num'] > 0).sum()}")
print(f"Stations with Level 2: {(df_feb_public['EV Level2 EVSE Num'] > 0).sum()}")
print(f"Stations with DC Fast: {(df_feb_public['EV DC Fast Count'] > 0).sum()}")

print("\n\nPublic Stations - Top States:")
print(df_feb_public['State'].value_counts().head(10))

Total stations: 65134
Public stations: 61308 (94.1%)
Private stations: 3826 (5.9%)


Public Stations - Charger Type Distribution:
Stations with Level 1: 179
Stations with Level 2: 53304
Stations with DC Fast: 9192


Public Stations - Top States:
State
CA    15626
NY     3764
FL     3234
TX     3060
MA     2892
WA     2092
CO     2087
GA     1908
PA     1655
MD     1617
Name: count, dtype: int64


In [8]:
# Save cleaned February 2024 dataset to CSV for use in dashboard
output_path = 'ev_charging_stations_feb2024_cleaned.csv'

# Select relevant columns for the dashboard
columns_to_save = [
    'Station Name', 'Street Address', 'City', 'State', 'ZIP',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
    'EV Network', 'EV Connector Types', 'Access Code', 'Facility Type',
    'Total_Chargers', 'Has_Level1', 'Has_Level2', 'Has_DC_Fast'
]

df_feb_to_save = df_feb[columns_to_save].copy()
df_feb_to_save.to_csv(output_path, index=False)

print(f"February 2024 cleaned dataset saved to: {output_path}")
print(f"Rows: {len(df_feb_to_save)}")
print(f"Columns: {len(df_feb_to_save.columns)}")
print(f"\nColumns saved: {list(df_feb_to_save.columns)}")

# Also save public stations only version
df_feb_public_to_save = df_feb_public[columns_to_save].copy()
df_feb_public_to_save.to_csv('ev_charging_stations_feb2024_public.csv', index=False)
print(f"\nFebruary 2024 public stations saved to: ev_charging_stations_feb2024_public.csv")
print(f"Rows: {len(df_feb_public_to_save)}")


February 2024 cleaned dataset saved to: ev_charging_stations_feb2024_cleaned.csv
Rows: 65134
Columns: 16

Columns saved: ['Station Name', 'Street Address', 'City', 'State', 'ZIP', 'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Network', 'EV Connector Types', 'Access Code', 'Facility Type', 'Total_Chargers', 'Has_Level1', 'Has_Level2', 'Has_DC_Fast']

February 2024 public stations saved to: ev_charging_stations_feb2024_public.csv
Rows: 61308


In [9]:
# Load and clean January 2023 dataset
ev_charging_station_jan = os.path.join(path, 'EV_Charging_Stations_Jan312023.xlsx')
df_jan = pd.read_excel(ev_charging_station_jan)

print("January 2023 Dataset Loaded")
print(f"Shape: {df_jan.shape}")

# Apply same cleaning steps
df_jan['EV Level1 EVSE Num'] = df_jan['EV Level1 EVSE Num'].fillna(0)
df_jan['EV Level2 EVSE Num'] = df_jan['EV Level2 EVSE Num'].fillna(0)
df_jan['EV DC Fast Count'] = df_jan['EV DC Fast Count'].fillna(0)
df_jan['Total_Chargers'] = df_jan['EV Level1 EVSE Num'] + df_jan['EV Level2 EVSE Num'] + df_jan['EV DC Fast Count']
df_jan['Has_Level1'] = df_jan['EV Level1 EVSE Num'] > 0
df_jan['Has_Level2'] = df_jan['EV Level2 EVSE Num'] > 0
df_jan['Has_DC_Fast'] = df_jan['EV DC Fast Count'] > 0

# Save cleaned January dataset
df_jan_to_save = df_jan[columns_to_save].copy()
df_jan_to_save.to_csv('ev_charging_stations_jan2023_cleaned.csv', index=False)

print(f"\nJanuary 2023 cleaned dataset saved to: ev_charging_stations_jan2023_cleaned.csv")
print(f"Rows: {len(df_jan_to_save)}")


January 2023 Dataset Loaded
Shape: (54238, 13)

January 2023 cleaned dataset saved to: ev_charging_stations_jan2023_cleaned.csv
Rows: 54238


In [10]:
# Display the cleaned February 2024 dataframe
print("Cleaned February 2024 Dataset Preview:")
df_feb.head(10)


Cleaned February 2024 Dataset Preview:


,Station Name,Street Address,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network,EV Connector Types,Access Code,Access Detail Code,Facility Type,Total_Chargers,Has_Level1,Has_Level2,Has_DC_Fast
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,0.0,57.0,2.0,SHELL_RECHARGE,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY,59.0,False,True,True
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,0.0,7.0,0.0,Non-Networked,J1772,public,NaN,PARKING_GARAGE,7.0,False,True,False
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,0.0,338.0,12.0,Non-Networked,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY,350.0,False,True,True
3,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,0.0,19.0,1.0,Non-Networked,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY,20.0,False,True,True
4,LADWP - Harbor Generating Station,161 N Island Ave,Wilmington,CA,90744,0.0,10.0,0.0,Non-Networked,J1772,private,NaN,UTILITY,10.0,False,True,False
5,LADWP - Sylmar West,13201 Sepulveda Blvd,Sylmar,CA,91342,0.0,2.0,0.0,Non-Networked,J1772,private,GOVERNMENT,UTILITY,2.0,False,True,False
6,LADWP - EV Service Center,1630 N Main St,Los Angeles,CA,90012,0.0,46.0,1.0,Non-Networked,CHADEMO J1772,private,NaN,UTILITY,47.0,False,True,True
7,LADWP - Fairfax Center,2311 S Fairfax Ave,Los Angeles,CA,90016,0.0,13.0,0.0,Non-Networked,J1772,private,NaN,UTILITY,13.0,False,True,False
8,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,0.0,3.0,0.0,Non-Networked,J1772,public,NaN,STATE_GOV,3.0,False,True,False
9,LADWP - Palmetto Center,1212 Palmetto St,Los Angeles,CA,90013,0.0,27.0,0.0,Non-Networked,J1772,private,NaN,UTILITY,27.0,False,True,False
